In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("../../")

#spectrome modules
from spectrome.utils import functions, path
from spectrome.brain import Brain
import scipy.stats as stats
from scipy.stats import friedmanchisquare


from spectrome.forward import ntf_local_stimulus as nt_stim
from spectrome.forward import ntf_local as nt_nostim
from spectrome.forward import ntf_local_stimulus as nt_stim

from scipy.io import loadmat
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import fdrcorrection

import pickle as pkl

In [2]:
##all the sessions data of MEG
data1=loadmat("../../../Database_PWelch_AvgTrials_MI_Baseline_RS1_19Subjects_MEG_Sess1.mat")
data2=loadmat("../../../Database_PWelch_AvgTrials_MI_Baseline_RS1_19Subjects_MEG_Sess2.mat")
data3=loadmat("../../../Database_PWelch_AvgTrials_MI_Baseline_RS1_19Subjects_MEG_Sess3.mat")
data4=loadmat("../../../Database_PWelch_AvgTrials_MI_Baseline_RS1_19Subjects_MEG_Sess4.mat")


##all the sessions mi 
sess1_mi_avg=data1['Avg_MI_MEG_DK']
sess2_mi_avg=data2['Avg_MI_MEG_DK']
sess3_mi_avg=data3['Avg_MI_MEG_DK']
sess4_mi_avg=data4['Avg_MI_EEG_DK']

##all the sessions resting state
sess1_rest_avg=data1['Avg_RS1_MEG_DK']
sess2_rest_avg=data2['Avg_RS1_MEG_DK']
sess3_rest_avg=data3['Avg_RS1_MEG_DK']
sess4_rest_avg=data4['Avg_RS1_EEG_DK']

##all the sessions baseline
sess1_base_avg=data1['Avg_Baseline_MEG_DK']
sess2_base_avg=data2['Avg_Baseline_MEG_DK']
sess3_base_avg=data3['Avg_Baseline_MEG_DK']
sess4_base_avg=data4['Avg_Baseline_EEG_DK']


In [ ]:
df=pd.read_excel('../../../spectrome-bci/spectrome/ListROIs_DK.xlsx')
fvec=np.squeeze(data1["Freqs"])[3:30]

In [4]:
all_regions=[]
all_regions_idx=[]
for i,r in df.iterrows():
    all_regions_idx.append(i)
    all_regions.append(r['ROI'].strip("'"))

In [5]:
##modeled data
x_mi_all={}
x_base_all={}
x_rest_all={}

for num in range(1, 5):
    x_mi_all[num]=np.zeros((68, 10, 19))
    x_base_all[num]=np.zeros((68, 10, 19))
    x_rest_all[num]=np.zeros((68, 10, 19))
    
    for i in range(19):
        mi_fname = f'../bci_results_apd/bci_results/meg/sess{num}/mi_allsubs_f/mi_nostim_mse_db_fvec330_{i}.p'
        with open(mi_fname, 'rb') as f:
            x_mi_all[num][:,:,i]=np.array(pkl.load(f))

        base_fname = f'../bci_results_apd/bci_results/meg/sess{num}/baseline_allsubs_f/baseline_nostim_mse_db_fvec330_{i}.p'
        with open(base_fname, 'rb') as f:
            x_base_all[num][:,:,i] = np.array(pkl.load(f))
        
        rest_fname = f'../bci_results_apd/bci_results/meg/sess{num}/rs1_allsubs_f0/rs1_nostim_mse_db_fvec330_{i}.p'
        with open(rest_fname, 'rb') as f:
            x_rest_all[num][:,:,i]=np.array(pkl.load(f))

### (MI-Rest)/Rest

In [7]:
##gei
sess1_mi_rest_gei=[[] for _ in range(68)]
sess2_mi_rest_gei=[[] for _ in range(68)]
sess3_mi_rest_gei=[[] for _ in range(68)]
sess4_mi_rest_gei=[[] for _ in range(68)]

 
for k in range(68):
  for j in range(19):
        mi_rest1_gei=(abs(x_mi_all[1][k,0,j])-abs(x_rest_all[1][k,0,j]))/abs(x_rest_all[1][k,0,j])
        sess1_mi_rest_gei[k].append(mi_rest1_gei)

        mi_rest2_gei=(abs(x_mi_all[2][k,0,j])-abs(x_rest_all[2][k,0,j]))/abs(x_rest_all[2][k,0,j])
        sess2_mi_rest_gei[k].append(mi_rest2_gei)

        mi_rest3_gei=(abs(x_mi_all[3][k,0,j])-abs(x_rest_all[3][k,0,j]))/abs(x_rest_all[3][k,0,j])
        sess3_mi_rest_gei[k].append(mi_rest3_gei)
        
        mi_rest4_gei=(abs(x_mi_all[4][k,0,j])-abs(x_rest_all[4][k,0,j]))/abs(x_rest_all[4][k,0,j])
        sess4_mi_rest_gei[k].append(mi_rest4_gei)

In [13]:
# for i in range(68):
#     if len(sess1_mi_rest_gei[i]) != 15:
#         print(i)

In [9]:
print("Incase of gei, significant regions are---")
for i in range(68):
    result = friedmanchisquare(sess1_mi_rest_gei[i], sess2_mi_rest_gei[i], sess3_mi_rest_gei[i], sess4_mi_rest_gei[i])
    if result[1]<0.05:
        for j, reg in enumerate(all_regions):
            if i==j:
                print(f"index {i}, {all_regions[j]}, {result}")


Incase of gei, significant regions are---
index 9, entorhinal R, FriedmanchisquareResult(statistic=10.005405405405407, pvalue=0.018520243373910412)


In [10]:
p_values = []

for i in range(68):
    result = friedmanchisquare(sess1_mi_rest_gei[i], sess2_mi_rest_gei[i], sess3_mi_rest_gei[i], sess4_mi_rest_gei[i])
    p_values.append(result[1])

fdr_corrected_pvals = fdrcorrection(p_values)[1]
fdr_corr = fdr_corrected_pvals.tolist()

for i in range(len(fdr_corr)):
    if fdr_corr[i] < 0.1:
        print(f"index {i}, {all_regions[i]}, {fdr_corr[i]}")

In [11]:
fdr_corrected_pvals

array([0.78170587, 0.79096724, 0.89161412, 0.84158303, 0.99000992,
       0.79995452, 0.79096724, 0.78170587, 0.79096724, 0.78170587,
       0.78170587, 0.79995452, 0.78170587, 0.79096724, 0.79096724,
       0.79096724, 0.7952096 , 0.79096724, 0.84766076, 0.78170587,
       0.85945888, 0.79096724, 0.79096724, 0.84766076, 0.79096724,
       0.79096724, 0.9484041 , 0.79096724, 0.99000992, 0.84158303,
       0.99000992, 0.99000992, 0.79096724, 0.99000992, 0.99000992,
       0.99000992, 0.83623298, 0.79995452, 0.79995452, 0.79096724,
       0.99000992, 0.79096724, 0.79995452, 0.78170587, 0.79096724,
       0.79096724, 0.79995452, 0.78170587, 0.79995452, 0.84158303,
       0.7952096 , 0.79995452, 0.78170587, 0.79995452, 0.79995452,
       0.79096724, 0.7952096 , 0.99000992, 0.78170587, 0.78170587,
       0.99000992, 0.79096724, 0.79995452, 0.79096724, 0.7952096 ,
       0.79096724, 0.79096724, 0.86709746])

In [12]:
##gii
sess1_mi_rest_gii=[[] for _ in range(68)]
sess2_mi_rest_gii=[[] for _ in range(68)]
sess3_mi_rest_gii=[[] for _ in range(68)]
sess4_mi_rest_gii=[[] for _ in range(68)]

 
for k in range(68):
    for j in range(19):
        mi_rest1_gii=((x_mi_all[1][k,1,j])-(x_rest_all[1][k,1,j]))/(x_rest_all[1][k,1,j])
        sess1_mi_rest_gii[k].append(mi_rest1_gii)
        mi_rest2_gii=((x_mi_all[2][k,1,j])-(x_rest_all[2][k,1,j]))/(x_rest_all[2][k,1,j])
        sess2_mi_rest_gii[k].append(mi_rest2_gii)
        mi_rest3_gii=((x_mi_all[3][k,1,j])-(x_rest_all[3][k,1,j]))/(x_rest_all[3][k,1,j])
        sess3_mi_rest_gii[k].append(mi_rest3_gii)
        mi_rest4_gii=((x_mi_all[4][k,1,j])-(x_rest_all[4][k,1,j]))/(x_rest_all[4][k,1,j])
        sess4_mi_rest_gii[k].append(mi_rest4_gii)


In [14]:
for i in range(68):
    result = friedmanchisquare(sess1_mi_rest_gii[i], sess2_mi_rest_gii[i], sess3_mi_rest_gii[i], sess4_mi_rest_gii[i])
    # print(result[1])
    if result[1]<0.05:
        for j, reg in enumerate(all_regions):
            if i==j:
                print(f"index {i}, {all_regions[j]}, {result}")

In [15]:
p_values = []

for i in range(68):
    result = friedmanchisquare(sess1_mi_rest_gii[i], sess2_mi_rest_gii[i], sess3_mi_rest_gii[i], sess4_mi_rest_gii[i])
    p_values.append(result[1])

fdr_corrected_pvals = fdrcorrection(p_values)[1]
fdr_corr = fdr_corrected_pvals.tolist()

for i in range(len(fdr_corr)):
    if fdr_corr[i] < 0.1:
        print(f"index {i}, {all_regions[i]}, {fdr_corr[i]}")

In [16]:
fdr_corrected_pvals

array([0.88784721, 0.88784721, 0.88784721, 0.96128261, 0.88784721,
       0.92640112, 0.90196146, 0.90366084, 0.90366084, 0.88784721,
       0.88784721, 0.88784721, 0.88784721, 0.88784721, 0.88784721,
       0.88784721, 0.88784721, 0.88784721, 0.88784721, 0.88784721,
       0.88784721, 0.88784721, 0.88784721, 0.99008941, 0.88784721,
       0.88784721, 0.90366084, 0.98162641, 0.88784721, 0.88784721,
       0.88784721, 0.95029669, 0.88784721, 0.88784721, 0.88784721,
       0.88784721, 0.96855329, 0.99008941, 0.88784721, 0.88784721,
       0.99008941, 0.88784721, 0.88784721, 0.95029669, 0.88784721,
       0.88784721, 0.88784721, 0.88784721, 0.99008941, 0.93755744,
       0.88784721, 0.88784721, 0.88784721, 0.88784721, 0.88784721,
       0.88784721, 0.88784721, 0.88784721, 0.88784721, 0.88784721,
       0.99008941, 0.88784721, 0.88784721, 0.93788986, 0.88784721,
       0.88784721, 0.88784721, 0.88784721])

In [17]:
##tau_e
sess1_mi_rest_tau_e=[[] for _ in range(68)]
sess2_mi_rest_tau_e=[[] for _ in range(68)]
sess3_mi_rest_tau_e=[[] for _ in range(68)]
sess4_mi_rest_tau_e=[[] for _ in range(68)]

 
for k in range(68):
    for j in range(19):
        mi_rest1_tau_e=((x_mi_all[1][k,2,j])-(x_rest_all[1][k,2,j]))/(x_rest_all[1][k,2,j])
        sess1_mi_rest_tau_e[k].append(mi_rest1_tau_e)
        mi_rest2_tau_e=((x_mi_all[2][k,2,j])-(x_rest_all[2][k,2,j]))/(x_rest_all[2][k,2,j])
        sess2_mi_rest_tau_e[k].append(mi_rest2_tau_e)
        mi_rest3_tau_e=((x_mi_all[3][k,2,j])-(x_rest_all[3][k,2,j]))/(x_rest_all[3][k,2,j])
        sess3_mi_rest_tau_e[k].append(mi_rest3_tau_e)
        mi_rest4_tau_e=((x_mi_all[4][k,2,j])-(x_rest_all[4][k,2,j]))/(x_rest_all[4][k,2,j])
        sess4_mi_rest_tau_e[k].append(mi_rest4_tau_e)

In [19]:
print("Incase of tau_e, significant regions are---")
for i in range(68):
    result = friedmanchisquare(sess1_mi_rest_tau_e[i], sess2_mi_rest_tau_e[i], sess3_mi_rest_tau_e[i], sess4_mi_rest_tau_e[i])
    if result[1]<0.05:
        for j, reg in enumerate(all_regions):
            if i==j:
                print(f"index {i}, {all_regions[j]}, {result}")


Incase of tau_e, significant regions are---
index 17, inferiortemporal R, FriedmanchisquareResult(statistic=7.941176470588244, pvalue=0.047243183430928426)
index 31, middletemporal R, FriedmanchisquareResult(statistic=9.609890109890136, pvalue=0.02219059747469413)
index 33, paracentral R, FriedmanchisquareResult(statistic=9.634920634920661, pvalue=0.021938513471930935)
index 51, precuneus R, FriedmanchisquareResult(statistic=8.460317460317492, pvalue=0.03739727630985676)


In [20]:
p_values = []

for i in range(68):
    result = friedmanchisquare(sess1_mi_rest_tau_e[i], sess2_mi_rest_tau_e[i], sess3_mi_rest_tau_e[i], sess4_mi_rest_tau_e[i])
    p_values.append(result[1])

fdr_corrected_pvals = fdrcorrection(p_values)[1]
fdr_corr = fdr_corrected_pvals.tolist()

for i in range(len(fdr_corr)):
    if fdr_corr[i] < 0.1:
        print(f"index {i}, {all_regions[i]}, {fdr_corr[i]}")

In [21]:
fdr_corrected_pvals

array([0.94955159, 0.94955159, 0.94955159, 0.94955159, 0.94955159,
       0.94955159, 0.94955159, 0.94758322, 0.94955159, 0.94955159,
       0.94955159, 0.94955159, 0.94955159, 0.94955159, 0.94955159,
       0.94955159, 0.94955159, 0.80313412, 0.94955159, 0.99727545,
       0.94955159, 0.94955159, 0.94955159, 0.94955159, 0.97838657,
       0.94955159, 0.94955159, 0.94955159, 0.99727545, 0.94955159,
       0.94758322, 0.75448031, 0.94955159, 0.75448031, 0.94955159,
       0.96368055, 0.99727545, 0.94955159, 0.99727545, 0.97838657,
       0.94955159, 0.94955159, 0.94955159, 0.97838657, 0.99727545,
       0.94955159, 0.97838657, 0.94955159, 0.94955159, 0.99727545,
       0.94955159, 0.80313412, 0.94955159, 0.94955159, 0.94955159,
       0.96519277, 0.94955159, 0.97838657, 0.97838657, 0.94955159,
       0.94955159, 0.94955159, 0.94955159, 0.94955159, 0.97838657,
       0.94758322, 0.94955159, 0.94955159])

In [22]:
##tau_i
sess1_mi_rest_tau_i=[[] for _ in range(68)]
sess2_mi_rest_tau_i=[[] for _ in range(68)]
sess3_mi_rest_tau_i=[[] for _ in range(68)]
sess4_mi_rest_tau_i=[[] for _ in range(68)]

 

for k in range(68):
   for j in range(19):
        mi_rest1_tau_i=((x_mi_all[1][k,3,j])-(x_rest_all[1][k,3,j]))/(x_rest_all[1][k,3,j])
        sess1_mi_rest_tau_i[k].append(mi_rest1_tau_i) 
        mi_rest2_tau_i=((x_mi_all[2][k,3,j])-(x_rest_all[2][k,3,j]))/(x_rest_all[2][k,3,j])
        sess2_mi_rest_tau_i[k].append(mi_rest2_tau_i)
        mi_rest3_tau_i=((x_mi_all[3][k,3,j])-(x_rest_all[3][k,3,j]))/(x_rest_all[3][k,3,j])
        sess3_mi_rest_tau_i[k].append(mi_rest3_tau_i)
        mi_rest4_tau_i=((x_mi_all[4][k,3,j])-(x_rest_all[4][k,3,j]))/(x_rest_all[4][k,3,j])
        sess4_mi_rest_tau_i[k].append(mi_rest4_tau_i)

In [23]:
print("Incase of tau_i, significant regions are---")
for i in range(68):
    result = friedmanchisquare(sess1_mi_rest_tau_i[i], sess2_mi_rest_tau_i[i], sess3_mi_rest_tau_i[i], sess4_mi_rest_tau_i[i])
    if result[1]<0.05:
        for j, reg in enumerate(all_regions):
            if i==j:
                print(f"index {i}, {all_regions[j]}, {result}")


Incase of tau_i, significant regions are---
index 29, medialorbitofrontal R, FriedmanchisquareResult(statistic=10.273743016759788, pvalue=0.016376961969895458)
index 34, parahippocampal L, FriedmanchisquareResult(statistic=10.989130434782645, pvalue=0.01178479686193941)
index 39, parsorbitalis R, FriedmanchisquareResult(statistic=8.569060773480677, pvalue=0.03560462440657816)
index 47, posteriorcingulate R, FriedmanchisquareResult(statistic=8.619047619047628, pvalue=0.03480901812379384)
index 52, rostralanteriorcingulate L, FriedmanchisquareResult(statistic=11.317460317460316, pvalue=0.010127459258647662)
index 62, supramarginal L, FriedmanchisquareResult(statistic=8.657608695652211, pvalue=0.034207165692828)


In [24]:
p_values = []

for i in range(68):
    result = friedmanchisquare(sess1_mi_rest_tau_i[i], sess2_mi_rest_tau_i[i], sess3_mi_rest_tau_i[i], sess4_mi_rest_tau_i[i])
    p_values.append(result[1])

fdr_corrected_pvals = fdrcorrection(p_values)[1]
fdr_corr = fdr_corrected_pvals.tolist()

for i in range(len(fdr_corr)):
    if fdr_corr[i] < 0.1:
        print(f"index {i}, {all_regions[i]}, {fdr_corr[i]}")

In [25]:
np.array(p_values),fdr_corrected_pvals

(array([0.65747904, 0.24958415, 0.39415963, 0.70653784, 0.24738202,
        0.52310787, 0.24723686, 0.15142183, 0.74767959, 0.07903731,
        0.51661591, 0.37184346, 0.50144638, 0.1706199 , 0.69398814,
        0.70828005, 0.73252812, 0.0977558 , 0.59307061, 0.16633893,
        0.66162213, 0.05594088, 0.09747375, 0.24910205, 0.77250906,
        0.19849066, 0.51983355, 0.71748301, 0.78564619, 0.01637696,
        0.48552963, 0.89789984, 0.65357704, 0.7372469 , 0.0117848 ,
        0.67672757, 0.48595812, 0.63111213, 0.90879759, 0.03560462,
        0.41206243, 0.31831668, 0.94576728, 0.37215561, 0.71578988,
        0.97582548, 0.41195021, 0.03480902, 0.26323978, 0.97293544,
        0.72592669, 0.45457515, 0.01012746, 0.59273341, 0.75055245,
        0.50034665, 0.85416181, 0.60165578, 0.37717315, 0.24000975,
        0.98358223, 0.69348221, 0.03420717, 0.14816148, 0.174544  ,
        0.10760365, 0.86640668, 0.18035955]),
 array([0.87995804, 0.73790095, 0.87995804, 0.87995804, 0.73790095,
  